## Final Project Submission

Please fill out:
* Student name: Aly (Aleigha Sardina-Spevack)
* Student pace: self paced
* Scheduled project review date/time: 1/29/2019 @ 6:30pm EST
* Instructor name: Eli 
* Blog post URL: https://medium.com/@sardina.aleigha/my-first-data-science-project-efce1e399893



# Housing Prices in Kings County

## Can we build a model to predict the housing prices in Kings County, Seattle?

### Of course the answer to that question is 'Yes'!  It's why we're here.  So we'll go through, end to end, a data science model using the OSEMN framework to arrive at a formula to help predict the home prices.


## Starting off...

Here are the column name definitions.  It's a good idea to keep these nearby as a reference.  It'll help us conceptualize what we are really working on.

### Column Names and descriptions for Kings County Data Set
* **id** - unique identified for a house
* **dateDate** - house was sold
* **pricePrice** -  is prediction target
* **bedroomsNumber** -  of Bedrooms/House
* **bathroomsNumber** -  of bathrooms/bedrooms
* **sqft_livingsquare** -  footage of the home
* **sqft_lotsquare** -  footage of the lot
* **floorsTotal** -  floors (levels) in house
* **waterfront** - House which has a view to a waterfront
* **view** - Has been viewed
* **condition** - How good the condition is ( Overall )
* **grade** - overall grade given to the housing unit, based on King County grading system
* **sqft_above** - square footage of house apart from basement
* **sqft_basement** - square footage of the basement
* **yr_built** - Built Year
* **yr_renovated** - Year when house was renovated
* **zipcode** - zip
* **lat** - Latitude coordinate
* **long** - Longitude coordinate
* **sqft_living15** - The square footage of interior housing living space for the nearest 15 neighbors
* **sqft_lot15** - The square footage of the land lots of the nearest 15 neighbors

# A bit about OSEMN

OSEMN stands for Obtain, Scrub, Explore, Model, & iNterpret and this acronym provides us with the steps that we will take to create our Housing Price predictor.  It's a popular model and provides a good framework for working through a problem like this.  

## First things first...
A few libraries as well as the data itself must be loaded into the notebook.  

In [1]:
#Library imports and using the magic key for inline plotting in matplot
#this won't be everything that we need, but it's a good slection to start with
#later on as we need more specific tools we can add them at that time.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
%matplotlib inline 

## This is our 'Obtain' step

In this case, we already have a .csv file ready to go.  Life isn't always this easy, but today it is so we can use pandas to import our full dataset.

In [2]:
#loading .csv dataset into a dataframe
df = pd.read_csv('kc_house_data.csv')
#viewing header to ensure it loaded properly
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,...,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,...,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,...,6,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,...,7,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,...,8,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


Here is where we start poking around a little to see what sort of data we have and getting a general of idea of the types of questions we need to ask. 

In [41]:
#we're not going to need the ID column so we can remove it right away
df = df.drop('id',axis=1)
df.head()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,10/13/2014,221900.0,3,1.00,1180,5650,1.0,0.0,0.0,3,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,3,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,3,6,770,0.0,1933,0.0,98028,47.7379,-122.233,2720,8062
3,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,5,7,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,3,8,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


In [42]:
#getting a feel for the data by reviewing column names and looking at some info related to the values in each column/series
#defining a column name veriable now so that we an use it later
col_name = list(df.columns)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 20 columns):
date             21597 non-null object
price            21597 non-null float64
bedrooms         21597 non-null int64
bathrooms        21597 non-null float64
sqft_living      21597 non-null int64
sqft_lot         21597 non-null int64
floors           21597 non-null float64
waterfront       21597 non-null float64
view             21597 non-null float64
condition        21597 non-null int64
grade            21597 non-null int64
sqft_above       21597 non-null int64
sqft_basement    21597 non-null float64
yr_built         21597 non-null int64
yr_renovated     21597 non-null float64
zipcode          21597 non-null int64
lat              21597 non-null float64
long             21597 non-null float64
sqft_living15    21597 non-null int64
sqft_lot15       21597 non-null int64
dtypes: float64(9), int64(10), object(1)
memory usage: 3.3+ MB


## Data Cleaning or Scrubbing

Hmmm... looks a bit messy.  We can see why we need to 'scrub' around in here 


After a little bit of exploring, it looks like we have some 'dirt' in our data that we'll need to clean up.  We can start with converting date to a datetime type.  This will make performing any furture operations on the date easier. Waterfront data seems to be missing some values, as does year renovated and view.  Then we have a text column for sqft_basement which doesn't match our other column with above grade sqft.  We'll walk through each of these one by one to see how we can tidy up a bit. 

In [43]:
df.date.astype('datetime64')

0       2014-10-13
1       2014-12-09
2       2015-02-25
3       2014-12-09
4       2015-02-18
5       2014-05-12
6       2014-06-27
7       2015-01-15
8       2015-04-15
9       2015-03-12
10      2015-04-03
11      2014-05-27
12      2014-05-28
13      2014-10-07
14      2015-03-12
15      2015-01-24
16      2014-07-31
17      2014-05-29
18      2014-12-05
19      2015-04-24
20      2014-05-14
21      2014-08-26
22      2014-07-03
23      2014-05-16
24      2014-11-20
25      2014-11-03
26      2014-06-26
27      2014-12-01
28      2014-06-24
29      2015-03-02
           ...    
21567   2014-06-10
21568   2014-12-02
21569   2014-08-28
21570   2014-10-15
21571   2015-03-05
21572   2014-11-13
21573   2014-09-10
21574   2014-05-14
21575   2014-10-02
21576   2015-04-16
21577   2015-03-17
21578   2014-10-17
21579   2014-10-31
21580   2014-08-13
21581   2015-04-21
21582   2014-10-13
21583   2014-09-15
21584   2014-10-15
21585   2015-04-07
21586   2014-06-26
21587   2014-08-25
21588   2015

In [44]:
#reviewing our basement data
df.sqft_basement.describe()

count    21597.000000
mean       285.716581
std        439.819830
min          0.000000
25%          0.000000
50%          0.000000
75%        550.000000
max       4820.000000
Name: sqft_basement, dtype: float64

In [45]:
#Checking to see how many unique values there are.  
#this could tell us if there is something about the data that may need to be fixed
df.sqft_basement.unique()

array([   0.,  400.,  910., 1530.,  730., 1700.,  300.,  970.,  760.,
        720.,  700.,  820.,  780.,  790.,  330., 1620.,  360.,  588.,
       1510.,  410.,  990.,  600.,  560.,  550., 1000., 1600.,  500.,
       1040.,  880., 1010.,  240.,  265.,  290.,  800.,  540.,  710.,
        840.,  380.,  770.,  480.,  570., 1490.,  620., 1250., 1270.,
        120.,  650.,  180., 1130.,  450., 1640., 1460., 1020., 1030.,
        750.,  640., 1070.,  490., 1310.,  630., 2000.,  390.,  430.,
        850.,  210., 1430., 1950.,  440.,  220., 1160.,  860.,  580.,
       2060., 1820., 1180.,  200., 1150., 1200.,  680.,  530., 1450.,
       1170., 1080.,  960.,  280.,  870., 1100.,  460., 1400.,  660.,
       1220.,  900.,  420., 1580., 1380.,  475.,  690.,  270.,  350.,
        935., 1370.,  980., 1470.,  160.,  950.,   50.,  740., 1780.,
       1900.,  340.,  470.,  370.,  140., 1760.,  130.,  520.,  890.,
       1110.,  150., 1720.,  810.,  190., 1290.,  670., 1800., 1120.,
       1810.,   60.,

Ah ha!  It looks like we've encountered a common problem.  A whole data series is represented as a string ebcause of one value.  In our case a '?'.  We should see how many '?'s there are before we move on.

In [46]:
df['sqft_basement'].value_counts()

0.0       13280
600.0       217
500.0       209
700.0       208
800.0       201
400.0       184
1000.0      148
900.0       142
300.0       142
200.0       105
750.0       104
450.0       103
530.0       103
480.0       103
720.0        98
620.0        90
580.0        84
840.0        83
420.0        81
860.0        79
670.0        78
1100.0       78
780.0        76
550.0        76
650.0        75
240.0        74
680.0        73
380.0        73
850.0        72
360.0        72
          ...  
2600.0        1
2610.0        1
276.0         1
274.0         1
1245.0        1
143.0         1
266.0         1
862.0         1
2180.0        1
415.0         1
243.0         1
1135.0        1
1525.0        1
3000.0        1
172.0         1
225.0         1
518.0         1
935.0         1
1920.0        1
1930.0        1
1990.0        1
588.0         1
1548.0        1
2390.0        1
602.0         1
915.0         1
295.0         1
1281.0        1
2130.0        1
906.0         1
Name: sqft_basement, Len

There are 452 ? and 12798 0s.  Seems like basements aren't popular in Kings County.  Let's change our ? to 0's since 0 is the mode of the dataset by a very large margin.

In [47]:
#changing ? to 0.0 so our data set reads as a floating point number, eventually.
df['sqft_basement'] = df['sqft_basement'].replace('?', '0.0')

In [48]:
#looking at our data series again 
df.sqft_basement.describe()

count    21597.000000
mean       285.716581
std        439.819830
min          0.000000
25%          0.000000
50%          0.000000
75%        550.000000
max       4820.000000
Name: sqft_basement, dtype: float64

In [49]:
df.sqft_basement.unique()

array([   0.,  400.,  910., 1530.,  730., 1700.,  300.,  970.,  760.,
        720.,  700.,  820.,  780.,  790.,  330., 1620.,  360.,  588.,
       1510.,  410.,  990.,  600.,  560.,  550., 1000., 1600.,  500.,
       1040.,  880., 1010.,  240.,  265.,  290.,  800.,  540.,  710.,
        840.,  380.,  770.,  480.,  570., 1490.,  620., 1250., 1270.,
        120.,  650.,  180., 1130.,  450., 1640., 1460., 1020., 1030.,
        750.,  640., 1070.,  490., 1310.,  630., 2000.,  390.,  430.,
        850.,  210., 1430., 1950.,  440.,  220., 1160.,  860.,  580.,
       2060., 1820., 1180.,  200., 1150., 1200.,  680.,  530., 1450.,
       1170., 1080.,  960.,  280.,  870., 1100.,  460., 1400.,  660.,
       1220.,  900.,  420., 1580., 1380.,  475.,  690.,  270.,  350.,
        935., 1370.,  980., 1470.,  160.,  950.,   50.,  740., 1780.,
       1900.,  340.,  470.,  370.,  140., 1760.,  130.,  520.,  890.,
       1110.,  150., 1720.,  810.,  190., 1290.,  670., 1800., 1120.,
       1810.,   60.,

In [50]:
#Casting our column as a string
df['sqft_basement'] = df['sqft_basement'].astype('float64')

In [51]:
df.sqft_basement.describe()

count    21597.000000
mean       285.716581
std        439.819830
min          0.000000
25%          0.000000
50%          0.000000
75%        550.000000
max       4820.000000
Name: sqft_basement, dtype: float64

Looking good.  We'll have a bit more to do before we can wrap our heads around what our data looks like.  This is, however, an iterative process and we should expect more tidying do to after this first round.


In [52]:
df.yr_renovated.unique()

array([   0., 1991., 2002., 2010., 1992., 2013., 1994., 1978., 2005.,
       2003., 1984., 1954., 2014., 2011., 1983., 1945., 1990., 1988.,
       1977., 1981., 1995., 2000., 1999., 1998., 1970., 1989., 2004.,
       1986., 2007., 1987., 2006., 1985., 2001., 1980., 1971., 1979.,
       1997., 1950., 1969., 1948., 2009., 2015., 1974., 2008., 1968.,
       2012., 1963., 1951., 1962., 1953., 1993., 1996., 1955., 1982.,
       1956., 1940., 1976., 1946., 1975., 1964., 1973., 1957., 1959.,
       1960., 1967., 1965., 1934., 1972., 1944., 1958.])

Let's take care of that nan and make it a 0.0.

In [53]:
#We'll change all the nan to 0
df['yr_renovated'] = df.yr_renovated.fillna(0.0)

In [54]:
df.yr_renovated.unique()

array([   0., 1991., 2002., 2010., 1992., 2013., 1994., 1978., 2005.,
       2003., 1984., 1954., 2014., 2011., 1983., 1945., 1990., 1988.,
       1977., 1981., 1995., 2000., 1999., 1998., 1970., 1989., 2004.,
       1986., 2007., 1987., 2006., 1985., 2001., 1980., 1971., 1979.,
       1997., 1950., 1969., 1948., 2009., 2015., 1974., 2008., 1968.,
       2012., 1963., 1951., 1962., 1953., 1993., 1996., 1955., 1982.,
       1956., 1940., 1976., 1946., 1975., 1964., 1973., 1957., 1959.,
       1960., 1967., 1965., 1934., 1972., 1944., 1958.])

In [55]:
#Change the 0 value to 'NaN'
df['yr_renovated'] = df.yr_renovated.replace('NaN', 0)

In [56]:
df.yr_renovated.unique()

array([   0., 1991., 2002., 2010., 1992., 2013., 1994., 1978., 2005.,
       2003., 1984., 1954., 2014., 2011., 1983., 1945., 1990., 1988.,
       1977., 1981., 1995., 2000., 1999., 1998., 1970., 1989., 2004.,
       1986., 2007., 1987., 2006., 1985., 2001., 1980., 1971., 1979.,
       1997., 1950., 1969., 1948., 2009., 2015., 1974., 2008., 1968.,
       2012., 1963., 1951., 1962., 1953., 1993., 1996., 1955., 1982.,
       1956., 1940., 1976., 1946., 1975., 1964., 1973., 1957., 1959.,
       1960., 1967., 1965., 1934., 1972., 1944., 1958.])

In [57]:
#checking the dataframe again to see what it looks like now
#this is a good habit to get into, jsut to make sure evrything is doign what we expect it to
#Spoiler!  You'll see an example of why father down
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 20 columns):
date             21597 non-null object
price            21597 non-null float64
bedrooms         21597 non-null int64
bathrooms        21597 non-null float64
sqft_living      21597 non-null int64
sqft_lot         21597 non-null int64
floors           21597 non-null float64
waterfront       21597 non-null float64
view             21597 non-null float64
condition        21597 non-null int64
grade            21597 non-null int64
sqft_above       21597 non-null int64
sqft_basement    21597 non-null float64
yr_built         21597 non-null int64
yr_renovated     21597 non-null float64
zipcode          21597 non-null int64
lat              21597 non-null float64
long             21597 non-null float64
sqft_living15    21597 non-null int64
sqft_lot15       21597 non-null int64
dtypes: float64(9), int64(10), object(1)
memory usage: 3.3+ MB


Much better!  Onto the waterfront series, let's see what that looks like.

In [58]:
df.waterfront.describe()

count    21597.000000
mean         0.006760
std          0.081944
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: waterfront, dtype: float64

In [59]:
df.waterfront.unique()

array([0., 1.])

Not what I expected!  It looks like a categorical data set.  Either the property is on the waterfront or it's not, or the data is not available.  The problem here is that it is stored as a floating point number.  This can lead to some improper inferences regarding our data. Luckily this isn't too complicated to fix.  Shall we?

In [60]:
#making our nan into 0's consistent
df['waterfront'] = df.waterfront.fillna('0.0')
#Casting our column as a string
df['waterfront'] = df['waterfront'].astype('float64')

In [61]:
#checking again to make sure this worked
df.waterfront.describe()

count    21597.000000
mean         0.006760
std          0.081944
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: waterfront, dtype: float64

In [62]:
df.waterfront.unique()

array([0., 1.])

In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 20 columns):
date             21597 non-null object
price            21597 non-null float64
bedrooms         21597 non-null int64
bathrooms        21597 non-null float64
sqft_living      21597 non-null int64
sqft_lot         21597 non-null int64
floors           21597 non-null float64
waterfront       21597 non-null float64
view             21597 non-null float64
condition        21597 non-null int64
grade            21597 non-null int64
sqft_above       21597 non-null int64
sqft_basement    21597 non-null float64
yr_built         21597 non-null int64
yr_renovated     21597 non-null float64
zipcode          21597 non-null int64
lat              21597 non-null float64
long             21597 non-null float64
sqft_living15    21597 non-null int64
sqft_lot15       21597 non-null int64
dtypes: float64(9), int64(10), object(1)
memory usage: 3.3+ MB


Better and better everytime.  We'll do a little bit more with that data series later, but for now we just have one more column to clean before we do some more exploring.

In [64]:
df.view.describe()

count    21597.000000
mean         0.233181
std          0.764673
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          4.000000
Name: view, dtype: float64

In [65]:
df.view.unique()

array([0., 3., 4., 2., 1.])

Oh look, another category.  Looks like we're missing some data as well.  First, let's handle this the same way as the waterfront tag.  Then we'll see about the missing data.

In [66]:
#making our nan into 0's consistent
df['view'] = df.view.fillna('0.0')
#Casting our column as a string
df['view'] = df['view'].astype('float64')

In [67]:
df.view.unique()

array([0., 3., 4., 2., 1.])

In [68]:
df.view.describe()

count    21597.000000
mean         0.233181
std          0.764673
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          4.000000
Name: view, dtype: float64

In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 20 columns):
date             21597 non-null object
price            21597 non-null float64
bedrooms         21597 non-null int64
bathrooms        21597 non-null float64
sqft_living      21597 non-null int64
sqft_lot         21597 non-null int64
floors           21597 non-null float64
waterfront       21597 non-null float64
view             21597 non-null float64
condition        21597 non-null int64
grade            21597 non-null int64
sqft_above       21597 non-null int64
sqft_basement    21597 non-null float64
yr_built         21597 non-null int64
yr_renovated     21597 non-null float64
zipcode          21597 non-null int64
lat              21597 non-null float64
long             21597 non-null float64
sqft_living15    21597 non-null int64
sqft_lot15       21597 non-null int64
dtypes: float64(9), int64(10), object(1)
memory usage: 3.3+ MB


## Let's have a look-see!

Our initial cleaning is done so let's have a look at the data.  In my opinion this is the exciting part: where we learn enough about the data to start asking some fun questions.

## Time to go Explore!

I don't know about you, but exploring the Seattle Metro sounds like fun to me.

In [70]:
for col in df.columns:
    print(col,'\n',df[col].describe(),'\n')

date 
 count         21597
unique          372
top       6/23/2014
freq            142
Name: date, dtype: object 

price 
 count    2.159700e+04
mean     5.402966e+05
std      3.673681e+05
min      7.800000e+04
25%      3.220000e+05
50%      4.500000e+05
75%      6.450000e+05
max      7.700000e+06
Name: price, dtype: float64 

bedrooms 
 count    21597.000000
mean         3.373200
std          0.926299
min          1.000000
25%          3.000000
50%          3.000000
75%          4.000000
max         33.000000
Name: bedrooms, dtype: float64 

bathrooms 
 count    21597.000000
mean         2.115826
std          0.768984
min          0.500000
25%          1.750000
50%          2.250000
75%          2.500000
max          8.000000
Name: bathrooms, dtype: float64 

sqft_living 
 count    21597.000000
mean      2080.321850
std        918.106125
min        370.000000
25%       1430.000000
50%       1910.000000
75%       2550.000000
max      13540.000000
Name: sqft_living, dtype: float64 

sqf

The above table tells us plenty of great information about housing in King County.  For example...

* The Mean price of a house sold is 540,296USD
* The Highest price of a house sold is 7,700,000USD
* The average number of bedrooms per home is 3, number of bathrooms is 2, with the largest home having 33 bedrooms!
* The average livable square feet is 2,080sqft, with the smallest home having only 370 sqft and the largest with over 13,500!

In [71]:
df['view'].value_counts()

0.0    19485
2.0      957
3.0      508
1.0      330
4.0      317
Name: view, dtype: int64

This tells us that 19,485 of the listed homes have not logged a viewing, where 317 have already been viewed 4 times.

We will start using some visualations 

In [ ]:
sns.countplot(df.bedrooms, order = df['bedrooms'].value_counts().index)

In [ ]:
#using the col_name variable to build a for lopp to plot some histograms of our data to get a feel for the distributions
#first we'll need to update col_name to get rid of the columns that contain only strings: it'll make our for loop throw an error
#we're leaving out yr_renovated since this only applies to a portion of our data.
col_name =['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 
           'condition', 'grade', 'sqft_above','sqft_basement','yr_built', 'zipcode', 
           'lat', 'long', 'sqft_living15', 'sqft_lot15']

for col in col_name:
    df[col].plot.hist( figsize= (6,6))
    plt.title(col)
    plt.show()

Yikes!  Most of these look like they have significant skew.  First we're going to take out floors and condition because they definitely aren't continuous variables.  Then we can try to noramlize these and add a kde plot to see if we can clean these up a bit and get a better picture.   I think we might have some additional non-continuous variables, but it'll be a clearer picture in a moment.

In [ ]:
#Once again we'll use a for loop to iterate through each of the columns 
continuous = ['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',  'grade',
              'sqft_above','sqft_basement','yr_built', 'zipcode', 'lat', 'long', 
              'sqft_living15', 'sqft_lot15']

for col in continuous:
    df[col].plot.hist(normed=True)
    df[col].plot.kde(label=col)
    plt.legend()
    plt.show()


My hunch was proved correct.  Look at the orange line for grade, bedroom and bathroom.  Clearly these are not continuous.  We should handle these categories by so our eventual regression will function properly.  

Before we get to all that we should check for multicollinearity to ensure our analysis will be readable and clear.  It may actually save us some code, by telling us that we can remove some variables that we would have needed to cleaned or managed in another way.  


In [ ]:
#let's look at a correlation matrix
df.corr()

This is a big matrix, let's simplify this a little.

In [ ]:
abs(df.corr()) > 0.75

That's a little better.  It looks like number of bathrooms, grade, and square feet above grade are  highly correlated with squarefeet of living space.  I think we can make this even clearer though.

Let's look at a heat map.

In [ ]:
sns.heatmap(df.corr(), center=0);

I think I see another pairs with high collinearity; square feet of living space of the nearest 15 neighbors.  I think the best thing to do would be to create a copy of our data frame and drop some of the columns that are highly correlated.

This revealed another surprise: latitude and longitude aren't correlated with zipcode. 

In [ ]:
#creating a dataframe copy
df_pred = df

#and as always checking out dataframe
df_pred.head()

In [ ]:
#dropping some correlated columns we're going to leave in square feet of the living space then get rid everything else.
#we are also going to drop some columns that essentially do the same thing 
for col in ['bathrooms','sqft_above','grade','sqft_living15']:
    df_pred.drop([col], axis=1, inplace = True)

In [ ]:
#checking our loop
df_pred.head()

Now that our data is even cleaner time to handle categorical varibles.  Let's look at the cleaned up version of the dataframe and review our KDE plots and histograms again.

In [ ]:
continuous2 = ['price', 'bedrooms', 'sqft_living', 'sqft_lot','sqft_basement','yr_built', 'zipcode', 'lat', 'long', 'sqft_lot15']

for col in continuous2:
    df[col].plot.hist(normed=True)
    df[col].plot.kde(label=col)
    plt.legend()
    plt.show()

Since we have categorical variables, we'll one-hot encode them.  The first one we'll look at is bedrooms.  Before we do that though, we'll import the encoder from scikit-learn to make our jobs easier.

In [ ]:
#importing an additional tool from a popular stats library SciKitLearn
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()

In [ ]:
#First we'll convert bedrooms to a category then we can tranform it into a one-hot encoded dataframe
beds = df.bedrooms.astype('category')
beds_dummies = lb.fit_transform(beds)
# converting to a dataframe
beds_dum_df = pd.DataFrame(beds_dummies, columns=lb.classes_)

In [ ]:
#checking our dataframe to ensure it worked
beds_dum_df.head()

33 bedrooms!  I wonder how many properties have 33 bedrooms.  That seems way like it might through our data off.  We can sum the column and if it's a low enough number we can remove it once we replace the columns in our actual dataframe.

In [ ]:
#Here we remove the original column and add back the dummy encoded columns
bed_dum = pd.get_dummies(df['bedrooms'],prefix='bed')
df_pred = df_pred.drop(['bedrooms'], axis=1)
df_pred= pd.concat([df_pred,bed_dum], axis=1)
#checking our dataframe
df_pred.head()

In [ ]:
df_pred['bed_33'].sum()

Only one.  Somehow, either a mega mansion or a multifamily property snuck into out data.  Out it goes!

In [ ]:
#removing the row that contains a one in our 'bed_33' column
df_pred.drop(df_pred[df_pred.bed_33 == 1].index, inplace=True)

In [ ]:
# first we'll check out the column sum to ensure it's gone, then delete the whole column
df_pred['bed_33'].sum()

In [ ]:
df_pred.drop(['bed_33'], axis=1, inplace=True)
df_pred.head()

We'll want to convert condition to a categorical variable as well.

In [ ]:
#Convering the next series to a category then we can tranform it into a one-hot encoded dataframe
condition = df.condition.astype('category')
condition_dummies = lb.fit_transform(condition)
#converting to a dataframe
condition_dum_df = pd.DataFrame(condition_dummies, columns=lb.classes_)
#Here we remove the original column and add back the dummy encoded columns
cond_dum = pd.get_dummies(df['condition'],prefix='condition')
df_pred = df_pred.drop(['condition'], axis=1)
df_pred= pd.concat([df_pred,cond_dum], axis=1)
#checking our dataframe
df_pred.head()

Now that we have cleaned our data, we can get to the fun stuff...


# The Questions!

Here is our chance to come up with some conclusions about our data set.  There are a few things about real estate that everyone always talks about, so now we can see if our data backs up all that talk

## What is the single biggest impact on housing price?
     Is it really about location?
## How much does the size of the property matter?
     Do we think it's more about the number of rooms, the lot size, or the square footage?
## What is the impact of having a property on the waterfront?  
     Is this a big factor in housing price?  Does it really increase price like I think it would? 
     

## Finally we're ready to begin Modeling.

We're going to start building our model.  We're going to put our categorical variables as the predictor in our model so we'll run a few (one for each categorical variable).

As you will notice, our OSEMN process is iterative.  You can really see how we think through things that way in the modelling step.

In [ ]:
#Creating a loop to look at each column in turn as a single predictor for housing price
col_names = df_pred.describe().columns.drop(['price','id'])
results=[['ind_var', 'r_squared','intercept', 'slope','p_value']]
for idx, val in enumerate(col_names):
    print("price~"+val)
    print("--------------------------")
    f = 'price~'+val
    model= smf.ols(formula =f, data=df_pred).fit()
    X_new = pd.DataFrame({val: [df_pred[val].min(), df_pred[val].max()]});
    preds= model.predict(X_new)
    results.append([val, model.rsquared, model.params[0], model.params[1], model.pvalues[1]])
    print(results[idx+1])

In [ ]:
pd.DataFrame(results)

Shoot! our R-Squared Values are low.  Let's see if we can fix that by transforming our target (price!) into a log of itself.

In [ ]:
#First we'll have a look at the log
np.log(df_pred['price']).describe()

In [ ]:
#converting that column of our dataframe to a log of itself and resaving that dataframe
pred_log = df_pred
pred_log['price'] = np.log(pred_log['price'])

In [ ]:
#looking at a histogram to see if taking the log improved the distibution
pred_log['price'].hist();

Whoa! what a difference!  Ok, so now we can repeat our OLS from above but this time using the log of price.

In [ ]:
col_names = pred_log.describe().columns.drop(['price','id'])
results_log=[['ind_var', 'r_squared','intercept', 'slope','p_value']]
for idx, val in enumerate(col_names):
    print("Housing price~"+val)
    print("--------------------------")
    f = 'price~'+val
    model= smf.ols(formula =f, data=pred_log).fit()
    X_new = pd.DataFrame({val: [pred_log[val].min(), pred_log[val].max()]});
    preds= model.predict(X_new)
    results_log.append([val, model.rsquared, model.params[0], model.params[1], model.pvalues[1]])
    print(results_log[idx+1])

In [ ]:
pd.DataFrame(results_log)

This looks much better.  This model has 25 variables though, and I think we can cut that down significantly.  We'll get rid of the poor predictors where R-Squared is less than 0.01 and we'll drop at least one column from each categorical variable.


In [ ]:
print(pred_log.columns)

In [ ]:
house_final = pred_log
house_final = house_final.drop(["id",'yr_built',"zipcode","long",'sqft_lot15','bed_11','condition_5'],axis=1)
house_final.describe()

In [ ]:
#data check to make sure something wasn't introduced when we were working on our dataset
house_final.isna().sum()

Ooops! We managed to get almost of a whole row with new na's.  I bet we can fix it by removing that row.  We can do this by setting the data frame equal to a single series that is all not null.  

Good thing we thought to check! 

In [ ]:
house_final = house_final[house_final.date.notnull()]

In [ ]:
house_final.isna().sum()

And to make sure we still have enough data we'll have another look at the dataframe statistics

In [ ]:
house_final.describe()

Looks Great!  

Starting here to get to the best fit model that would answer our questions (which we will return to shortly), we can use feature ranking with recursive feature elimination on our dataset.  But before we do that we will spilt our data into two chunks; one for training our model and one for testing our model.  

In [ ]:
#we have bunch of libraries and library features we will import here to do both our train-test split
#and to do the RFE(recursive feature elimination) for us; and finally importing the regression modeler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()

In [ ]:
#creating a matrix so that we can begin our rfe
#we have to take the date column out or our rfe function will throw errors
y = house_final[["price"]]
X = house_final.drop(['price', 'date'], axis=1)
X.head()

In [ ]:
#we're going to do an 20/80 split ; 20% of the data to test the model and the rest to train it
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.20, random_state = 3)

MSE_test=[]
MSE_train=[]
list_n= list(range(5,86,10))

for n in list_n:
    select_n = RFE(linreg, n_features_to_select = n)
    select_n = select_n.fit(X_train,np.ravel(y_train))
    selected_columns = X.columns[select_n.support_]
    linreg.fit(X_train[selected_columns],y_train)
    yhat_train = linreg.predict(X_train[selected_columns])
    yhat_test = linreg.predict(X_test[selected_columns])
    mse_train = np.sum((y_train - yhat_train)**2)/len(y_train)
    mse_test = np.sum((y_test - yhat_test)**2)/len(y_test)
    print(mse_train)
    print(mse_test)
    MSE_test.append(mse_test)
    MSE_train.append(mse_test)

That is too many variables to really get a good sense of how they affect our model.  Additionally, with all the categorical varibles on-hot encoded, some of the test data would have columns with almsot all 0s.  You can imagine how that could negatively affect the accuracy of our model.  Maybe we should move to a model with fewer predictors.

In [ ]:
#we're moving out of scikit and back to stats models so we'll need to import the relevant parts before we try agian
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [ ]:
#running our regression model
X_int = sm.add_constant(X)
model = sm.OLS(y,X_int.astype(float)).fit()
model.summary()

In [ ]:
model.params

The bedrooms columns when they get to higher numbers of bedrooms are very rare values and they are throwing off our mosel.  We should get rid of these predictors and rerun our model. 

In [ ]:
X = house_final.drop(['price', 'date', 'bed_1', 'bed_2', 'bed_3', 'bed_4', 
                      'bed_5', 'bed_6', 'bed_7', 'bed_8', 'bed_9', 'bed_10'], axis=1)

In [ ]:
X_int = sm.add_constant(X)
model = sm.OLS(y,X_int.astype(float)).fit()
model.summary()

## iNterpreting the model we created

The coefficients for the  model we just created can be interpreted as follows:  how does the log of price change for each additional unit of X.  Where the X can be our predictors: the squarefeet of the living space, whether there is a waterfront on the property or how many floors the home has.  It's a little more complicated than using price un-trasformed but it worked much better for our model. As it stands now, our model explains about 70% of the variation in the log(price).  

### Finalizing The Model

I think we have a lot of variables here.  We can pick our top three and rerun the model to see if we can explain a good protion of the change in our home price while keeping things a little simpler.  Then we can make a formula for our model, that we can use on real world data!

In [ ]:
#Another iteration of a regression
X = house_final.drop(['price', 'date', 'bed_1', 'bed_2', 'bed_3', 'bed_4', 
                      'bed_5', 'bed_6', 'bed_7', 'bed_8', 'bed_9', 'bed_10',
                     'sqft_basement', 'yr_renovated', 'sqft_lot','sqft_living',
                     'view', 'condition_1', 'condition_2', 'condition_3', 'condition_4'], axis=1)

In [ ]:
X_int = sm.add_constant(X)
model = sm.OLS(y,X_int.astype(float)).fit()
model.summary()

In [ ]:
model.params

There we have it! we can look at the model below and plug in our values and get a predicted value for our housing prices!

### log(price) = 0.279*floors + 1.093*waterfront + 1.660*latitude - 66.330 

# Back to the Questions!

## What is the single biggest impact on housing price?
     Is it really about location?
     
   Why yes it is!  Turns out, Latitude had a big part to play in the variation amongst different housing prices.  I'm rather surprised that zipcode didn't have a bigger impact than latitude. 
     
## How much does the size of the property matter?
     Do we think it's more about the number of rooms, the lot size, or the square footage?
     
   Seems that size also matters a lot.  In our final model, the number of floors was a good predictor.  In our previous, more complicated model, we had a number of predictors realted to size, so we can assume that it plays a big role both in general and more specifically as the number of floors.  
     
     
## What is the impact of having a property on the waterfront?  
     Is this a big factor in housing price?  Does it really increase price like I think it would? 
     
   Well, this answer is also yes!  When I asked myself this question I thought perhaps that I might find there were other factors that are more important.  I am actually surprised at how large the coefficient is, though.  I guess these properties are really *that much more* desireable.  (I can certainly say I would want one!)

## And there we have it!

We followed the OSEMN process to clean, model, and interpret the Kings County Dataset.  We saw that each step was iterative, and that even as we progessed through the model, we went back to previous steps and moved on again.  

What we learned about the date through exploration and through the model itself, we can bring back to the business users and help them some forecasts or predictions using our model.  

## And if we had more time..

There are a number of things I would add if I had more time to work on this data.

1. Adding a map can help visualize some of the conclusions we made about location.
2. Taking another look at the dates of the sales.  We could try to see if prices are really higher in the summer when more people are looking to buy
3. Reviewing out scaling process and trying a different method to see if it gives us better results